<a href="https://colab.research.google.com/github/Artinto/2023_AI_python_study/blob/main/%5BKDW%5DCNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 32

# MNIST Dataset
train_dataset = datasets.CIFAR10(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.CIFAR10(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
print(train_loader.dataset)

100%|██████████| 170498071/170498071 [00:02<00:00, 68463425.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data/
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [ ]:
for (X_train, Y_train) in  train_loader:
    print(f"X_train: {X_train.size()} type: {X_train.type()}")
    # X_train은 [batch size, channel, width, height]로 구성 
    # 여기서 channel은 R,G,B로 3개의 channel
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
Y_train: torch.Size([32]) type: torch.LongTensor


In [ ]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.mp = nn.MaxPool2d(2, stride = 2)
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))

        x = x.view(-1, 8 * 8 * 16)  # flatten the tensor
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = Net().to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(image), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for image, label in test_loader:
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        output = model(image)
        test_loss += criterion(output, label).item()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 30):
    train(epoch)
    test()

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.295376
Train Epoch: 1 [320/50000 (1%)]	Loss: 2.301405
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.342977
Train Epoch: 1 [960/50000 (2%)]	Loss: 2.257677
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.265126
Train Epoch: 1 [1600/50000 (3%)]	Loss: 2.213754
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.163881
Train Epoch: 1 [2240/50000 (4%)]	Loss: 2.200684
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.188128
Train Epoch: 1 [2880/50000 (6%)]	Loss: 2.060894
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.079305
Train Epoch: 1 [3520/50000 (7%)]	Loss: 2.066973
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.110434
Train Epoch: 1 [4160/50000 (8%)]	Loss: 2.086083
Train Epoch: 1 [4480/50000 (9%)]	Loss: 2.064315
Train Epoch: 1 [4800/50000 (10%)]	Loss: 1.962640
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.036973
Train Epoch: 1 [5440/50000 (11%)]	Loss: 2.046690
Train Epoch: 1 [5760/50000 (12%)]	Loss: 2.020478
Train Epoch: 1 [6080/50000 (12%)]	Loss: 2.111958
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2